<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [131]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [540]:
#!pip install hyperopt

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [31]:
!head -1 ../../../data/train.json > ../../../data/train1.json

In [347]:
# активирует вывод консольных сообщений на английском (вместо крокозябл из кириллистических символов)
!chcp 65001

Active code page: 65001


In [80]:
with open('../../../data/train1.json', encoding='utf-8') as inp_json:
    first_json = json.load(inp_json)

In [183]:
len(first_json)

16

In [81]:
first_json

{'_id': 'https://habrahabr.ru/post/7/',
 '_timestamp': 1493192186.0903192,
 'author': {'name': 'Павел Титов',
  'nickname': '@ptitov',
  'url': 'https://habrahabr.ru/users/ptitov'},
 'content': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'domain': 'habrahabr.ru',
 'flags': [],
 'flow': None,
 'hubs': [{'id': 'hub/habr',
   'title': 'Хабрахабр',
   'url': 'https://habrahabr.ru/hub/habr/'}],
 'link_tags': {'alternate': 'https://habrahabr.ru/rss/post/7/',
  'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
  'canonical': 'https://habrahabr.ru/post/7/',
  'icon': '/images/favicons/favicon-16x16.png',
  'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
  'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'},
 'meta_tags': {'al:android:app_name': 'Habrahabr',
  'al:an

In [33]:
first_json.keys()

dict_keys(['_id', '_timestamp', 'url', 'domain', 'published', 'title', 'content', 'polling', 'post_id', 'flags', 'hubs', 'flow', 'tags', 'author', 'link_tags', 'meta_tags'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [34]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [35]:
first_json['_timestamp']

1493192186.0903192

In [36]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [37]:
first_json['domain']

'habrahabr.ru'

In [38]:
first_json['published']

{'$date': '2006-07-15T01:48:00.000Z'}

экспериментируем со временем

In [132]:
datetime.strptime(first_json['published']['$date'], '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[3]

1

In [39]:
first_json['title']

'Самопроизвольное разлогинивание'

In [40]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [55]:
str(len(first_json['content']))

'199'

In [41]:
first_json['polling']

In [42]:
first_json['post_id']

7

In [43]:
first_json['flags']

[]

In [44]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [45]:
first_json['flow']

In [46]:
first_json['tags']

['логин', 'login']

In [47]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [48]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [49]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [52]:
train_target = pd.read_csv('../../../data/train_target.csv',
                          index_col='url')

In [441]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


In [258]:
# тест для "доставания" метки-значения из файла
train_target.loc[first_json['url'], 'target']

0.69314700000000007

Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов.
Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [23]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1
1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14


In [440]:
!head -2 ../../../data/habr_train2.vw

0.693147 |title РЎР°РјРѕРїСЂРѕРёР·РІРѕР»СЊРЅРѕРµ СЂР°Р·Р»РѕРіРёРЅРёРІР°РЅРёРµ |tags Р»РѕРіРёРЅ login |domain habrahabr.ru |flow None |author @ptitov |hubs РҐР°Р±СЂР°С…Р°Р±СЂ |num content_len:0.0 month:7 hour:1
1.098612 |title Stand-along cРѕРѕР±С‰РµСЃС‚РІР° РїСЂРѕС‚РёРІ СЃРѕРѕР±С‰РµСЃС‚РІ РІ СЂР°РјРєР°С… СЃРѕС†РёР°Р»СЊРЅС‹С… СЃРµС‚РµР№ |tags СЃРѕРѕР±С‰РµСЃС‚РІР° РёРЅС‚РµСЂРЅРµС‚-СЃРѕРѕР±С‰РµСЃС‚РІРѕ СЃРѕС†РёР°Р»СЊРЅС‹Рµ СЃРµС‚Рё РЅРёС€РµРІРѕР№ Р±СЂРµРЅРґ |domain geektimes.ru |flow None |author @AlexBruce |hubs Р§С‘СЂРЅР°СЏ РґС‹СЂР° |num content_len:0.0 month:7 hour:14


ПРИМЕЧАНИЕ: как видим, имеется проблемка с кодировкой при выводе на экран. Тем не менее в файле все ОК (читается нормально) и на результат это тоже не повлияло

In [498]:
# функция для подготовки ОБУЧАЮЩЕЙ выборки для формата vowpal wabbit
# не забываем убирать из текста ":" и "|"

def to_vw_format_train(document, target):
    return str(target.loc[document['url'], 'target']) \
                              + ' |title ' + str(document['title'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |tags ' + str(' '.join(document['tags']) or 'None').replace("|"," ").replace(":"," ") \
                              + ' |domain ' + str(document['domain'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |flow ' + str(document['flow'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |author ' + str(document['author']['nickname'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |hubs ' + str(document['hubs'][0]['title'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |num ' + 'content_len:' + str(round(len(document['content']) / 10**6, 2)) \
                                         + ' month:' + str(datetime.strptime(document['published']['$date'], 
                                                                             '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[1]) \
                                         + ' hour:' + str(datetime.strptime(document['published']['$date'], 
                                                                             '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[3]) \
                                + '\n'

Отлаживаем подготовку данных для VW

In [486]:
#  Проверка encoding в среде Python

#!python -c 
import sys; print(sys.stdout.encoding)

UTF-8


In [487]:
# тест: подготовка данных для VW
!head -500 ../../../data/train.json > ../../../data/train2.json

In [488]:
# тест (продолжение): подготовка данных для VW
with open('../../../data/train2.json', encoding='utf-8') as inp_json, \
     open('../../../data/habr_train2.vw', 'w', encoding='utf-8') as out_vw:
    #n = json.loads(inp_json)
    for line in tqdm_notebook(inp_json, total=500):
        data_json = json.loads(line)
        
        # Ваш код здесь
        out_vw.write(to_vw_format_train(data_json, train_target))

In [499]:
# подготовка train данных для VW

with open('../../../data/train.json', encoding='utf-8') as inp_json, \
     open('../../../data/habr_train.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json, total=120000):
        data_json = json.loads(line)
        
        # Ваш код здесь
        out_vw.write(to_vw_format_train(data_json, train_target))

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [500]:
# функция для подготовки ТЕСТОВОЙ выборки для формата vowpal wabbit
def to_vw_format_test(document):
    return '1' \
                              + ' |title ' + str(document['title'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |tags ' + str(' '.join(document['tags']) or 'None').replace("|"," ").replace(":"," ") \
                              + ' |domain ' + str(document['domain'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |flow ' + str(document['flow'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |author ' + str(document['author']['nickname'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |hubs ' + str(document['hubs'][0]['title'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |num ' + 'content_len:' + str(round(len(document['content']) / 10**6, 2)) \
                                         + ' month:' + str(datetime.strptime(document['published']['$date'], 
                                                                             '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[1]) \
                                         + ' hour:' + str(datetime.strptime(document['published']['$date'], 
                                                                             '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[3]) \
                                + '\n'

In [501]:
# подготовка test данных для VW
with open('../../../data/test.json', encoding='utf-8') as inp_json, \
     open('../../../data/habr_test.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json, total=52913):
        data_json = json.loads(line)
        
        # Ваш код здесь
        out_vw.write(to_vw_format_test(data_json))

посмотрим для контроля на 1-й url тестовой выборки

In [415]:
!head -1 ../../../data/test.json > ../../../data/test1.json
with open('../../../data/test1.json', encoding='utf-8') as inp_json:
    test1_json = json.load(inp_json)

In [416]:
test1_json['url']

'https://geektimes.ru/post/87455/'

In [343]:
!head -2 ../../../data/habr_test.vw

1 |title Р”РµРЅСЊ РџРё! |tags РџРё Pi |domain geektimes.ru |flow None |author @Timursan |hubs Р§С‘СЂРЅР°СЏ РґС‹СЂР° |num content_len:0.0 month:3 hour:3
1 |title РЎРєСЂРёРїС‚ РґР»СЏ СЂР°Р·Р±РёРµРЅРёСЏ РѕР±СЂР°Р·РѕРІ РјСѓР·С‹РєР°Р»СЊРЅС‹С… CD РЅР° С‚СЂРµРєРё Рё РєРѕРЅРІРµСЂС‚Р°С†РёРё РІ С„РѕСЂРјР°С‚ FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Р§С‘СЂРЅР°СЏ РґС‹СЂР° |num content_len:0.01 month:3 hour:0


In [24]:
!head -2 ../../data/habr_test.vw

1 |title День Пи! |tags Пи Pi |domain geektimes.ru |flow None |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3
1 |title Скрипт для разбиения образов музыкальных CD на треки и конвертации в формат FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

# Блок отладки обучения и валидации

In [285]:
# Ваш код здесь

# разбиваем train на обучающую и отложенную
# готовим адреса для разбивки: 70% - train, 30% - valid
n_total = 120000
n_train = int(n_total * 0.7)
n_valid = n_total - n_train
n_train, n_valid

(84000, 36000)

непосредственно разбиваем:

In [502]:
with open('../../../data/habr_train.vw', encoding='utf-8') as train, \
     open('../../../data/habr_work_train.vw', 'w', encoding='utf-8') as work_train, \
     open('../../../data/habr_work_valid.vw', 'w', encoding='utf-8') as work_valid:
    for i in range(n_train):
        work_train.write(train.readline())
    for i in range(n_valid):
        work_valid.write(train.readline())

Обучим Vowpal Wabbit:

In [493]:
%%time
!vw -d ../../../data/habr_work_train.vw -c --passes 3 -k \
    -f ../../../data/habr_model.vw

final_regressor = ../../../data/habr_model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ../../../data/habr_work_train.vw.cache
Reading datafile = ../../../data/habr_work_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.480453 0.480453            1            1.0   0.6931   0.0000       11
0.248073 0.015693            2            2.0   1.0986   0.9733       23
0.138161 0.028250            4            4.0   0.0000   0.1317       26
3.758118 7.378074            8            8.0   4.9698   0.6343       25
3.973398 4.188678           16           16.0   2.3026   1.4936       19
4.819290 5.665183           32           32.0   2.8904   3.0134       15
4.103258 3.387225           64           64.0   0.0000   3.0282       15
3.223466 2.343673          128          128.0   5.1475   2.1160       16
2.

Wall time: 5.1 s


Модель обучена.  Применим обученную модель на ОТЛОЖЕННОЙ выборке, сохраняя предсказания в файл с помощью опции -p:

In [494]:
!vw -i ../../../data/habr_model.vw -t -d ../../../data/habr_work_valid.vw \
-p ../../../data/habr_work_valid_predictions.txt

only testing
predictions = ../../../data/habr_work_valid_predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../../data/habr_work_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.573763 0.573763            1            1.0   1.3863   2.1438       14
0.578485 0.583208            2            2.0   0.6931   1.4568       19
0.539411 0.500336            4            4.0   0.0000   0.0000       23
0.691060 0.842709            8            8.0   1.3863   0.6253       17
0.520036 0.349012           16           16.0   1.6094   0.7845       12
0.649091 0.778146           32           32.0   2.5649   1.8322       22
1.048092 1.447093           64           64.0   4.2627   2.4291       24
1.096254 1.144416          128          128.0   0.0000   0.6062       21
1.088529 1.080803          256          256.0

### Организуем валидацию

валидационные значения

In [495]:
valid_vals = train_target.iloc[n_train:, 0].values

In [496]:
valid_vals.shape

(36000,)

валидационные предсказанные значения

In [504]:
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')

In [505]:
valid_predictions.shape

(36000,)

In [506]:
# 1-я валидация
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

0.95311521102777785

In [497]:
# 2-я валидация
valid_predictions = pd.read_csv('../../../data/habr_work_valid_predictions.txt', header=None).values
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

0.95413588913888892

вывод: при изменения округления длины контента с 2 до 3 метрика увеличилась. Возвращаем до 2

# Блок подбора параметров

### Отладка подбора и ручной подбор

In [503]:
# обучаем
!vw -d ../../../data/habr_work_train.vw -c --passes 3 -k \
    -f ../../../data/habr_model.vw \
    --quiet
# применяем на валидационной
!vw -i ../../../data/habr_model.vw -t -d ../../../data/habr_work_valid.vw \
    -p ../../../data/habr_work_valid_predictions.txt \
    --quiet

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

0.95311521102777785

In [512]:
# обучаем
!vw -d ../../../data/habr_work_train.vw -c -k \
    -f ../../../data/habr_model.vw \
    --quiet
# применяем на валидационной
!vw -i ../../../data/habr_model.vw -t -d ../../../data/habr_work_valid.vw \
    -p ../../../data/habr_work_valid_predictions.txt \
    --quiet

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

# убрал 3 прохода на обучении

0.94133603402777777

In [527]:
# обучаем
!vw -d ../../../data/habr_work_train.vw -c --passes 5 -k \
    -f ../../../data/habr_model.vw \
    --quiet
# применяем на валидационной
!vw -i ../../../data/habr_model.vw -t \
    -d ../../../data/habr_work_valid.vw \
    -p ../../../data/habr_work_valid_predictions.txt \
    --quiet

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

# 5 проходов

0.94671300052777774

In [529]:
# обучаем
!vw -d ../../../data/habr_work_train.vw -c --passes 5 -k \
    -f ../../../data/habr_model.vw \
    --quiet
# применяем на валидационной
!vw -i ../../../data/habr_model.vw -t \
    -d ../../../data/habr_work_valid.vw \
    -p ../../../data/habr_work_valid_predictions.txt \
    --quiet \
    --ngram title3

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

# 5 проходов
# ngram

0.94671300052777774

#### добавим в признак 'content'
- (не прошло: ошибки, отлаживать пока не стал - вернул обратно)

In [532]:
# функция для подготовки ОБУЧАЮЩЕЙ выборки для формата vowpal wabbit
# не забываем убирать из текста ":" и "|"

def to_vw_format_train(document, target):
    return str(target.loc[document['url'], 'target']) \
                              + ' |title ' + str(document['title'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |tags ' + str(' '.join(document['tags']) or 'None').replace("|"," ").replace(":"," ") \
                              + ' |domain ' + str(document['domain'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |flow ' + str(document['flow'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |author ' + str(document['author']['nickname'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |hubs ' + str(document['hubs'][0]['title'] or 'None').replace("|"," ").replace(":"," ") \
                              + ' |num ' + 'content_len:' + str(round(len(document['content']) / 10**6, 2)) \
                                         + ' month:' + str(datetime.strptime(document['published']['$date'], 
                                                                             '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[1]) \
                                         + ' hour:' + str(datetime.strptime(document['published']['$date'], 
                                                                             '%Y-%m-%dT%H:%M:%S.000Z').timetuple()[3]) \
                                + '\n'

#                               + ' |content' + str(document['content'] or 'None').replace("|"," ").replace(":"," ") \
                                
# подготовка train данных для VW

with open('../../../data/train.json', encoding='utf-8') as inp_json, \
     open('../../../data/habr_train.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json, total=120000):
        data_json = json.loads(line)
        
        # Ваш код здесь
        out_vw.write(to_vw_format_train(data_json, train_target))

        
# разбиваем на трейн и валидационную        
with open('../../../data/habr_train.vw', encoding='utf-8') as train, \
     open('../../../data/habr_work_train.vw', 'w', encoding='utf-8') as work_train, \
     open('../../../data/habr_work_valid.vw', 'w', encoding='utf-8') as work_valid:
    for i in range(n_train):
        work_train.write(train.readline())
    for i in range(n_valid):
        work_valid.write(train.readline())
        

# обучаем
!vw -d ../../../data/habr_work_train.vw -c --passes 5 -k \
    -f ../../../data/habr_model.vw \
    --quiet
# применяем на валидационной
!vw -i ../../../data/habr_model.vw -t \
    -d ../../../data/habr_work_valid.vw \
    -p ../../../data/habr_work_valid_predictions.txt \
    --quiet \
    --ngram title3

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

# 5 проходов
# ngram
# добавляем в признаки контент - удалили

0.94671300052777774

In [675]:
# обучаем
!vw \
-d ../../../data/habr_work_train.vw -c --passes 5 -k \
-f ../../../data/habr_model.vw \
--loss_function squared \
--learning_rate 0.19429722438207644 \
--power_t 0.1460283361704552 \
--ngram 2 \
--initial_t 0.38906999500560485 \
-b 26 \
--random_seed=17 \
--quiet
# применяем на валидационной
!vw \
-i ../../../data/habr_model.vw -t \
-d ../../../data/habr_work_valid.vw \
-p ../../../data/habr_work_valid_predictions.txt \
--quiet \

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

# 5 проходов
# ngram

0.89381156866666667

Бенчмарк, который Вы видите в соревновании (vw_baseline.csv) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), **биграммами** и настроенными гиперпараметрами **bits, learning_rate и power_t.**

## Подбор параметров с помощью модуля hyperopt

подробности здесь https://github.com/hyperopt/hyperopt/wiki/FMin

In [545]:
#!pip install hyperopt
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [589]:
best_results = {}

In [704]:
def hyperopt_objective(params):
    # обучаем
    !vw \
    -d ../../../data/habr_work_train.vw \
    -f ../../../data/habr_model.vw \
    -c \
    --passes 5 \
    -k \
    --loss_function squared \
    --learning_rate {params['learning_rate']} \
    --power_t {params['power_t']} \
    --ngram 2 \
    --initial_t {params['initial_t']} \
    -b 26 \
    --random_seed=17 \
    --quiet
    
    
    # применяем на валидационной
    !vw -i ../../../data/habr_model.vw \
        -t -d ../../../data/habr_work_valid.vw \
        -p ../../../data/habr_work_valid_predictions.txt \
        --quiet
        
        
    # scoring
    valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
    score_MAE = mean_absolute_error(valid_vals, valid_predictions)
    
    return score_MAE

# блок изменения параметров
params_space = {
    'learning_rate': hp.uniform('learning_rate', 0., 2.0), # default = 0.5
    'power_t': hp.uniform('power_t', 0., 1.0), # default = 0.5
    'initial_t': hp.uniform('initial_t', 0., 2.0), # default = 0
                }

In [705]:
%%time
# запускаем подбор
trials = Trials()

best_params = fmin(
    hyperopt_objective,
    space=params_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

Wall time: 18min 27s


In [706]:
trials.best_trial

{'book_time': datetime.datetime(2017, 11, 2, 18, 52, 2, 312000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'initial_t': [65], 'learning_rate': [65], 'power_t': [65]},
  'tid': 65,
  'vals': {'initial_t': [0.8083671309918706],
   'learning_rate': [1.763117182149858],
   'power_t': [0.10174824581435331]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2017, 11, 2, 18, 52, 17, 750000),
 'result': {'loss': 0.8865221409722223, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 65,
 'version': 0}

In [707]:
best_results[6] = trials.best_trial

In [708]:
best_results

{1: {'book_time': datetime.datetime(2017, 11, 2, 12, 51, 10, 919000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'l': [7], 'power_t': [7]},
   'tid': 7,
   'vals': {'l': [0.20252062086232894], 'power_t': [0.270693419194566]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2017, 11, 2, 12, 51, 14, 183000),
  'result': {'loss': 0.9325812230277776, 'status': 'ok'},
  'spec': None,
  'state': 2,
  'tid': 7,
  'version': 0},
 2: {'book_time': datetime.datetime(2017, 11, 2, 13, 46, 1, 740000),
  'exp_key': None,
  'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'idxs': {'initial_t': [11], 'l': [11], 'power_t': [11]},
   'tid': 11,
   'vals': {'initial_t': [0.048500245470919356],
    'l': [0.1889127730045205],
    'power_t': [0.19809558558023838]},
   'workdir': None},
  'owner': None,
  'refresh_time': datetime.datetime(2017, 11, 2, 13, 46, 5, 89000),
  'result': {'loss': 0.9304649387222222, 'status': 'ok

In [709]:
trials.argmin

{'initial_t': 0.8083671309918706,
 'learning_rate': 1.763117182149858,
 'power_t': 0.10174824581435331}

## Контрольная проверка перед формированием submission

In [710]:
# обучаем
!vw \
-d ../../../data/habr_work_train.vw \
-f ../../../data/habr_model.vw \
-c \
--passes 5 \
-k \
--loss_function squared \
--learning_rate {trials.argmin['learning_rate']} \
--power_t {trials.argmin['power_t']} \
--ngram 2 \
--initial_t {trials.argmin['initial_t']} \
-b 26 \
--random_seed=17 \
--quiet

# применяем на валидационной
!vw \
-i ../../../data/habr_model.vw -t \
-d ../../../data/habr_work_valid.vw \
-p ../../../data/habr_work_valid_predictions.txt \
--quiet

# scoring
valid_predictions = np.loadtxt('../../../data/habr_work_valid_predictions.txt')
score_MAE = mean_absolute_error(valid_vals, valid_predictions)
score_MAE

# 5 проходов
# -b 26
# --ngram 2 (как и 3)

0.88652214097222226

# Блок предсказания на ТЕСТЕ и формирования submission

Обучим модель на ПОЛНОЙ обучающей выборке с определенными наилучшими параметрами

In [711]:
!vw \
-d ../../../data/habr_train.vw \
-f ../../../data/habr_model.vw \
-c \
--passes 5 \
-k \
--loss_function squared \
--learning_rate {trials.argmin['learning_rate']} \
--power_t {trials.argmin['power_t']} \
--ngram 2 \
--initial_t {trials.argmin['initial_t']} \
-b 26 \
--random_seed=17 \
--quiet

Модель обучена.  Применим обученную модель на ТЕСТОВОЙ выборке, сохраняя предсказания в файл с помощью опции -p:

In [712]:
!vw \
-i ../../../data/habr_model.vw \
-t -d ../../../data/habr_test.vw \
-p ../../../data/habr_test_predictions.txt \
--quiet

In [713]:
sample_sub = pd.read_csv('../../../data/sample_submission.csv', 
                         index_col='url')
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [714]:
your_test_predictions = pd.read_csv('../../../data/habr_test_predictions.txt', header=None)
your_test_predictions.columns = ['target']
your_test_predictions.head()

,target
0,1.214495
1,3.255367
2,0.828338
3,3.264878
4,0.140983


In [715]:
your_submission = sample_sub.copy()
your_test_predictions.index = your_submission.index
your_submission['target'] = your_test_predictions['target']
your_submission.to_csv('../../../data/submission.csv')

your_submission.head()

,target
url,
https://geektimes.ru/post/87455/,1.214495
https://geektimes.ru/post/87452/,3.255367
https://geektimes.ru/post/87459/,0.828338
https://habrahabr.ru/post/87461/,3.264878
https://habrahabr.ru/post/5754/,0.140983


Для получения баллов в #mlcourse_open команда (из 1 человека) должна называться в точном соответствии с тем, как оно записано в рейтинге.